# 使用链式调用进行结构化输出推理的 o1 应用教程

初始发布的 o1 推理模型版本（2024 年 9 月）具备先进的能力，但尚未支持结构化输出。这意味着在使用 o1 时，无法可靠地获得类型安全的输出，模型依赖于提示本身返回有用的 JSON 数据。

在本教程中，我们将探讨两种方法，利用 o1 模型（特别是 `o1-preview`），在使用 OpenAI API 时返回有效的 JSON 格式。

## 目标

我们将通过一个示例来演示如何：

1. 获取公司的维基百科页面；
2. 确定哪些公司最能从 AI 技术中受益；
3. 将结果以 JSON 格式返回，便于其他系统进行处理。

## 方法一：直接提示

最简单的方法是明确地提示 `o1-preview` 返回 JSON 响应。

### 代码示例

In [3]:
import requests
from openai import OpenAI

client = OpenAI()


In [ ]:
import requests
from openai import OpenAI

client = OpenAI()

def fetch_html(url):
    response = requests.get(url)
    if response.status_code == 200:
        return response.text
    else:
        return None

url = "https://en.wikipedia.org/wiki/List_of_largest_companies_in_the_United_States_by_revenue"
html_content = fetch_html(url)

json_format = """
{
    companies: [
        {
            \"company_name\": \"OpenAI\",
            \"page_link\": \"https://en.wikipedia.org/wiki/OpenAI\",
            \"reason\": \"OpenAI would benefit because they are an AI company...\"
        }
    ]
}
"""

o1_response = client.chat.completions.create(
    model="o1-preview",
    messages=[
        {
            "role": "user", 
            "content": f"""
你是一位商业分析师，旨在了解 AI 技术如何应用于大型企业。

- 阅读以下 HTML 内容，并返回哪些公司可以从 AI 技术中受益：{html_content}。
- 通过比较这些公司，按机会大小进行排名，并展示排名前 3 的公司。仅以 JSON 格式返回，格式如下：{json_format}"
"""
        }
    ]
)

print(o1_response.choices[0].message.content)

### 输出结果

```json
{
    "companies": [
        {
            "company_name": "Walmart",
            "page_link": "https://en.wikipedia.org/wiki/Walmart",
            "reason": "Walmart 可以通过 AI 技术优化供应链管理、库存水平、提升客户服务（如 AI 聊天机器人）和提供个性化购物体验。AI 能帮助 Walmart 更准确地预测需求、降低运营成本并提高整体效率。"
        },
        {
            "company_name": "UnitedHealth Group",
            "page_link": "https://en.wikipedia.org/wiki/UnitedHealth_Group",
            "reason": "UnitedHealth Group 可以利用 AI 技术改进患者护理，通过预测分析个性化治疗方案、检测欺诈性索赔并简化行政流程。AI 能协助早期疾病检测、提高诊断准确性并增强数据分析以获得更好的健康结果。"
        },
        {
            "company_name": "Ford Motor Company",
            "page_link": "https://en.wikipedia.org/wiki/Ford_Motor_Company",
            "reason": "福特汽车公司可以通过 AI 技术推进自动驾驶车辆的开发、利用自动化和机器人技术优化制造流程、实施预测性维护，并通过 AI 驱动的功能提升车内体验。AI 能帮助福特提高安全性、降低生产成本并创新新的交通解决方案。"
        }
    ]
}
```

### 说明

上述响应已经相当不错，返回了具有适当响应的 JSON 数据。然而，这种仅依赖提示的 JSON 推断方法也存在一些问题：

- 需要手动处理 JSON 数据以适配类型安全的结构；
- 模型的拒绝（如无法处理的请求）不会明确地在 API 中作为单独的结构返回。

## 方法二：使用结构化输出

现在，我们将使用结构化输出功能。为此，我们将把 `o1-preview` 的响应与对 `gpt-4o-mini` 的后续请求链接起来，后者可以有效地处理从初始 `o1-preview` 响应返回的数据。

### 代码示例

In [10]:
import requests
from openai import OpenAI

client = OpenAI()

def fetch_html(url):
    response = requests.get(url)
    if response.status_code == 200:
        return response.text
    else:
        return None

url = "https://en.wikipedia.org/wiki/List_of_Nobel_Memorial_Prize_laureates_in_Economic_Sciences"
html_content = fetch_html(url)

In [11]:
html_content

'<!DOCTYPE html>\n<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-1 vector-feature-appearance-pinned-clientpref-1 vector-feature-night-mode-enabled skin-theme-clientpref-day vector-toc-available" lang="en" dir="ltr">\n<head>\n<meta charset="UTF-8">\n<title>List of Nobel Memorial Prize laureates in Economic Sciences - Wikipedia</title>\n<script>(function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled 

In [15]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def fetch_html(url):
    response = requests.get(url)
    if response.status_code == 200:
        return response.text
    else:
        return None

def extract_table(url):
    html_content = fetch_html(url)
    if not html_content:
        print("无法获取网页内容")
        return None

    soup = BeautifulSoup(html_content, 'lxml')
    table = soup.find('table', {'class': 'wikitable sortable'})
    
    if not table:
        print("未找到指定的表格。")
        return None

    df = pd.read_html(str(table))[0]
    return df

def main():
    url = "https://en.wikipedia.org/wiki/List_of_Nobel_Memorial_Prize_laureates_in_Economic_Sciences"
    df = extract_table(url)
    
    if df is not None:
        print("表格前5行数据：")
        print(df.head())

        # 保存为CSV文件
        df.to_csv("nobel_economics_laureates_full.csv", index=False, encoding='utf-8-sig')
        print("表格已保存为 'nobel_economics_laureates_full.csv'")
    else:
        print("表格提取失败。")


表格前5行数据：
   Year  Portrait      Laureate (birth/death)         Country  \
0  1969       NaN   Ragnar Frisch (1895–1973)          Norway   
1  1969       NaN   Jan Tinbergen (1903–1994)     Netherlands   
2  1970       NaN  Paul Samuelson (1915–2009)   United States   
3  1971       NaN   Simon Kuznets (1901–1985)   United States   
4  1972       NaN      John Hicks (1904–1989)  United Kingdom   

                                           Rationale  \
0  "for having developed and applied dynamic mode...   
1  "for having developed and applied dynamic mode...   
2  "for the scientific work through which he has ...   
3  "for his empirically founded interpretation of...   
4  "for their pioneering contributions to general...   

  PhD (or equivalent) alma mater  \
0             University of Oslo   
1              Leiden University   
2             Harvard University   
3            Columbia University   
4           University of Oxford   

  Institution (most significant tenure/at time

In [7]:
from pydantic import BaseModel
from typing import List, Optional

class NobelLaureateEducation(BaseModel):
   """诺贝尔获奖者教育经历模型"""
   phd_university: str  # 博士学位授予大学
   main_institution: str  # 主要任职机构 
   academic_pattern: str  # 学术发展模式(如:本土发展/国际流动等)

class NobelLaureateContribution(BaseModel):
   """诺贝尔获奖者研究贡献模型"""
   key_theories: List[str]  # 关键理论贡献列表
   research_field: str  # 研究领域
   impact_area: str  # 影响范围/领域

class NobelLaureate(BaseModel):
   """单个诺贝尔获奖者完整信息模型"""
   year: int  # 获奖年份
   name: str  # 获奖者姓名
   life_span: str  # 生命跨度(出生-死亡年份)
   country: str  # 国籍
   rationale: str  # 获奖理由
   education: NobelLaureateEducation  # 教育经历
   contribution: NobelLaureateContribution  # 研究贡献

class NobelLaureatesData(BaseModel):
   """诺贝尔获奖者数据集合模型"""
   laureates: List[NobelLaureate]  # 获奖者列表


# 首先读取CSV的前两行数据
import pandas as pd

df = pd.read_csv("nobel_economics_laureates_full.csv", encoding='utf-8-sig')
sample_data = df.head(1).to_string()
sample_data

'   Year  Portrait     Laureate (birth/death) Country                                                                                    Rationale PhD (or equivalent) alma mater Institution (most significant tenure/at time of receipt)                  Key contributions (non-exhaustive)\n0  1969       NaN  Ragnar Frisch (1895–1973)  Norway  "for having developed and applied dynamic models for the analysis of economic processes"[2]             University of Oslo                                       University of Oslo  Frisch–Waugh–Lovell theorem, Conjectural variation'

In [8]:

o1_response = client.chat.completions.create(
    model="o1-preview",
    messages=[
        {
            "role": "user", 
            "content": f"""
你是一位专业的学术分析师，专注于研究学术发展路径和研究影响力分析。

请阅读以下诺贝尔经济学奖得主数据（前两位获奖者）：

{sample_data}

请从以下维度分析这些数据：

1. 学术发展模式分析：
   - 教育背景特点
   - 职业发展路径
   - 地理流动特征

2. 研究影响力分析：
   - 主要理论贡献
   - 研究领域特点
   - 实际应用价值

请为每位获奖者提供结构化分析，包含以下字段：
- 获奖者姓名
- 学术发展特征
- 研究贡献亮点
- 影响力评估
- 现代意义

最后，请总结这些早期诺贝尔经济学奖得主的共同特征和差异点。

分析时请注重数据背后的深层含义，而不是简单的数据复述。
"""
        }
    ]
)

o1_response_content = o1_response.choices[0].message.content

In [9]:
o1_response_content

'**一、拉格纳·弗里希（Ragnar Frisch）**\n\n**获奖者姓名：**\n拉格纳·弗里希（1895–1973）\n\n**学术发展特征：**\n\n- **教育背景特点：**\n  弗里希毕业于奥斯陆大学，主修经济学和数学。他对统计学和数学方法有深厚的兴趣，这为他后来的计量经济学研究奠定了坚实的基础。\n\n- **职业发展路径：**\n  他在奥斯陆大学任教，成为经济学教授。他是计量经济学的创始人之一，致力于将数学和统计学方法应用于经济学分析。\n\n- **地理流动特征：**\n  弗里希主要在挪威工作，但也与国际学者保持合作。他曾多次访问其他国家，参加国际会议，促进了计量经济学的国际化发展。\n\n**研究贡献亮点：**\n\n- **主要理论贡献：**\n  - 创立了“计量经济学”这一术语，推动了经济学的数量化研究。\n  - 提出了弗里希-沃夫-洛夫尔定理，这是回归分析中的重要理论。\n  - 发展了动态经济模型，分析经济过程的动态特性。\n\n- **研究领域特点：**\n  弗里希的研究主要集中在计量经济学、经济动态模型和统计方法的应用上。他注重方法论的严谨性，强调理论与实证的结合。\n\n- **实际应用价值：**\n  他的工作为经济学提供了强有力的工具，使得经济理论可以通过统计方法进行检验和应用，对政策分析和经济预测产生了深远影响。\n\n**影响力评估：**\n\n弗里希被誉为计量经济学之父，他的工作奠定了现代经济学实证研究的基础。他的发展动态模型方法为经济周期和政策效果的分析提供了重要手段，对后世经济学家产生了重大影响。\n\n**现代意义：**\n\n在当代经济学中，计量经济学已成为不可或缺的研究工具。弗里希的理论和方法仍然被广泛应用于经济预测、政策评估和金融分析等领域，为学术研究和实际应用提供了重要支持。\n\n---\n\n**二、扬·廷伯根（Jan Tinbergen）**\n\n**获奖者姓名：**\n扬·廷伯根（1903–1994）\n\n**学术发展特征：**\n\n- **教育背景特点：**\n  廷伯根在莱顿大学获得物理学博士学位。他的理科背景使他具备了严谨的数量分析能力，之后转向经济学研究。\n\n- **职业发展路径：**\n  - 任职于荷兰中央统计局，从事经济统计工作。\n  - 在荷兰经济学院（现

In [10]:
from pprint import pprint
response = client.beta.chat.completions.parse(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "user", 
            "content": f"""
根据以下数据，使用给定的响应格式进行格式化：{o1_response_content}
"""
        }
    ],
    response_format=NobelLaureatesData,
)

pprint(response.choices[0].message.parsed)

NobelLaureatesData(laureates=[NobelLaureate(year=1969, name='Ragnar Frisch', life_span='1895–1973', country='Norway', rationale='For their development of econometric methods and their application to the analysis of economic processes.', education=NobelLaureateEducation(phd_university='University of Oslo', main_institution='University of Oslo', academic_pattern='Economics and Mathematics'), contribution=NobelLaureateContribution(key_theories=["Founded the term 'econometrics'", 'Proposed the Frisch-Waugh-Lovell theorem', 'Developed dynamic economic models'], research_field='Econometrics, Economic Dynamics, Statistical Methods', impact_area='Provided tools for empirical testing of economic theories affecting policy analysis and forecasting')), NobelLaureate(year=1974, name='Jan Tinbergen', life_span='1903–1994', country='Netherlands', rationale='For having developed and applied dynamic models for the analysis of economic processes.', education=NobelLaureateEducation(phd_university='Leiden

https://platform.openai.com/docs/guides/structured-outputs/introduction?context=without_parse

### 输出结果

In [ ]:
NobelLaureatesData(
    laureates=[
        NobelLaureate(
            year=1969,
            name='Ragnar Frisch',
            life_span='1895–1973',
            country='Norway',
            rationale='For their development of econometric methods and their application to the analysis of economic processes.',
            education=NobelLaureateEducation(
                phd_university='University of Oslo',
                main_institution='University of Oslo',
                academic_pattern='Economics and Mathematics'
            ),
            contribution=NobelLaureateContribution(
                key_theories=[
                    "Founded the term 'econometrics'",
                    'Proposed the Frisch-Waugh-Lovell theorem',
                    'Developed dynamic economic models'
                ],
                research_field='Econometrics, Economic Dynamics, Statistical Methods',
                impact_area='Provided tools for empirical testing of economic theories affecting policy analysis and forecasting'
            )
        ),
        NobelLaureate(
            year=1974,
            name='Jan Tinbergen',
            life_span='1903–1994',
            country='Netherlands',
            rationale='For having developed and applied dynamic models for the analysis of economic processes.',
            education=NobelLaureateEducation(
                phd_university='Leiden University',
                main_institution='Erasmus University Rotterdam',
                academic_pattern='Physics and later Economics'
            ),
            contribution=NobelLaureateContribution(
                key_theories=[
                    'Developed econometric models',
                    "Introduced 'Tinbergen's Rule'",
                    'Pioneered economic policy modeling'
                ],
                research_field='Macroeconomic Modeling, Economic Policy Analysis, Development Economics',
                impact_area='Provided quantitative tools for economic policy formulation, particularly for developing economies'
            )
        )
    ]
)

### 说明

通过这种方法，我们：

- 定义了数据模型 `CompanyData` 和 `CompaniesData`，以确保类型安全；
- 首先使用 `o1-preview` 获取初始响应；
- 然后使用 `gpt-4o-mini` 解析初始响应，并将其格式化为我们定义的结构化数据格式。

## 总结

结构化输出使您的代码能够实现可靠的类型安全和更简单的提示。此外，它允许您重用对象模式，便于集成到现有的工作流程中。

当前，o1 系列模型尚未支持结构化输出，但我们可以通过将两个请求链接在一起，重用 `gpt-4o-mini` 的结构化输出功能。虽然此流程需要两个调用，但第二个 `gpt-4o-mini` 调用的成本相对于 `o1-preview` 或 `o1-mini` 的调用成本应是最小的。

## 结语

通过本教程，您应该已经了解了如何使用链式调用的方法，在使用 o1 模型时实现结构化输出。这种方法不仅提高了数据处理的可靠性，还简化了数据集成的过程。希望您能将此方法应用到实际项目中，充分发挥 o1 模型的强大功能。